In [1]:
# 01 - chat dialog
import guidance
import openai
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 35.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import re
# guidance.llm = guidance.llms.OpenAI("gpt-4")
# vicuna = guidance.llms.transformers.Vicuna("TheBloke/stable-vicuna-13B-HF", device_map="auto")
# vicuna = guidance.llms.transformers.Vicuna("TheBloke/stable-vicuna-13B-GPTQ", device_map="auto")
# vicuna = guidance.llms.transformers.Vicuna("TheBloke/vicuna-7B-1.1-HF", device_map="auto")
vicuna = guidance.llms.transformers.Vicuna("TheBloke/Wizard-Vicuna-7B-Uncensored-HF", device_map="auto")
guidance.llm = vicuna

/workspace/venv/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:26<00:00, 13.03s/it]


### Multi-step chat

In [3]:
def parse_best(prosandcons, options):
    best = re.search(r'Best=(\d+)', prosandcons)
    if not best:
        best =  re.search(r'Best.*?(\d+)', 'Best= option is 3')
    if best:
        best = int(best.group(1))
    else:
        best = 0
    return options[best]


In [4]:
create_plan = guidance('''{{#system~}}
You are a helpful assistant.
{{~/system}}
{{#block hidden=True}}
{{#user~}}
I want to {{goal}}.
{{~! generate potential options ~}}
Can you please generate one option for how to accomplish this?
Please make the option very short, at most one line.
{{~/user}}
{{#assistant~}}
{{gen 'options' n=5 temperature=1.0 max_tokens=600}}
{{~/assistant}}
{{/block}}
{{~! generate pros and cons and select the best option ~}}
{{#block hidden=True}}
{{#user~}}
I want to {{goal}}.

Can you please comment on the pros and cons of each of the following options, and then pick the best option?
---{{#each options}}
Option {{@index}}: {{this}}{{/each}}
---
Please discuss each option very briefly (one line for pros, one for cons), and end by saying Best=X, where X is the best option.
{{~/user}}
{{#assistant~}}
{{gen 'prosandcons' temperature=0.0 max_tokens=600}}
{{~/assistant}}
{{/block}}
{{#user~}}
I want to {{goal}}.
{{~! Create a plan }}
Here is my plan:
{{parse_best prosandcons options}}
Please elaborate on this plan, and tell me how to best accomplish it.
{{~/user}}
{{#assistant~}}
{{gen 'plan' max_tokens=500}}
{{~/assistant}}''')



In [5]:
out = create_plan(goal='read more books', parse_best=parse_best)

Stop program system You are a helpful assistant. user I want to read more books . 
Here is my plan:
 Read every day. 
Please elaborate on this plan, and tell me how to best accomplish it. assistant Great plan! Here are some tips to help you read more books:
1. Set a goal: Decide how many books you want to read in a certain time frame (e.g. a month, a quarter, a year).
2. Make time: Find a consistent time each day to read, even if it's just 15-20 minutes.
3. Create a reading list: Make a list of books you want to read and prioritize them based on your interests.
4. Use technology: There are many apps and websites that can help you track your reading progress and recommend books based on your preferences.
5. Join a book club: Joining a book club can help you stay accountable and motivated to read more.
6. Don't get discouraged: It's normal to have days where you don't feel like reading, but don't give up! Keep trying and you'll find a rhythm that works for you.